In [1]:
# Note: this notebook is build on Python 3.9 ver.
# snscrape developer version

!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git

In [2]:
import snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd

from textblob import TextBlob
import re
from emoji import UNICODE_EMOJI

This is the complementary analysis performed due to the fact that LDA topic modeling didn't work well. Here I just scrape the tweets and perform some simple sentiment analysis and calculations, the tweets data are not saved. 

### Marcus Rashford

In [3]:
# the 2021–22 Premier League start from 2021-08-13

l=[]
for i in pd.date_range('2021-08-13',periods=161, freq='D'):
    l.append(str(i)[0:10])
l[-1]

'2022-01-20'

In [4]:
def get_tweet(query,start,end):

    search = query + ' -filter:retweets + since:' + start + ' until:' + end + ' lang:en'

    result = []
    for ind,tweet in enumerate(sntwitter.TwitterSearchScraper(search).get_items()): 
        result.append([tweet.content])
    
    return result

def clean_tweet(txt): 
    temp = re.sub("@[A-Za-z0-9_]+","", txt)
    temp1 = re.sub("#[A-Za-z0-9_]+","", temp)
    temp2 = re.sub(r"http\S+", "", temp1)
    result = ''.join(i for i in temp2 if i not in UNICODE_EMOJI['en'])
    return result

def sentiment(cleaned_tweet):
    blob = TextBlob(cleaned_tweet)
    return blob.sentiment.polarity

    
def sentiment_label(score):
    if score >= 0.75:
        return 'POSITIVE'
    elif score >= 0.2:
        return 'positive'
    elif score >= -0.2:
        return 'neutral'
    elif score >= -0.75:
        return 'negative'
    else:
        return 'NEGATIVE'

In [5]:
tweets_list = []

for i in range(len(l)-1):
    start=l[i]
    end=l[i+1]
    tweets= get_tweet('MarcusRashford', start, end)
    tweets_list += tweets

df = pd.DataFrame(tweets_list, columns=['Text'])
df.shape

(193377, 1)

In [6]:
df['clean_text']= df.apply(lambda row: clean_tweet(row['Text']), axis=1)
df['sentiment_score']= df.apply(lambda row: sentiment(row['clean_text']), axis=1)
df['senti_label']= df.apply(lambda row: sentiment_label(row['sentiment_score']), axis=1)

df.head()

,Text,clean_text,sentiment_score,senti_label
0,@Negbawi @AndrewMMacLeod @Purna_Sen @GodivaRid...,...,0.160000,neutral
1,@MarcusRashford @Nike @MarcusRashford I would ...,I would Stand next to you Any Day for all o...,0.428571,positive
2,@YanitedAlways @MarcusRashford 👀 Should ask hi...,Should ask him and if he says no dig up dir...,0.000000,neutral
3,@MarcusRashford Get well soon bro,Get well soon bro,0.000000,neutral
4,@YanitedAlways @MarcusRashford Got any spare t...,Got any spare time?,0.000000,neutral


In [7]:
df.sentiment_score.mean()

0.1263575103577549

In [8]:
result = df.groupby('senti_label').count()['Text']
result = result.reset_index()
total = df.shape[0]

print('There are',total, ' tweets overall during the span about Marcus Rashford.')

for i in range(5):
    value = result['Text'][i]
    label = result['senti_label'][i]
    rate = round(100*value/total,3)
    print(value, 'tweets (',rate, '% ) are labeled as',label)

There are 193377  tweets overall during the span about Marcus Rashford.
1774 tweets ( 0.917 % ) are labeled as NEGATIVE
13895 tweets ( 7.185 % ) are labeled as POSITIVE
13552 tweets ( 7.008 % ) are labeled as negative
117030 tweets ( 60.519 % ) are labeled as neutral
47126 tweets ( 24.37 % ) are labeled as positive


### Cristiano Ronaldo

In [9]:
tweets_list = []

for i in range(len(l)-1):
    start=l[i]
    end=l[i+1]
    tweets= get_tweet('Cristiano Ronaldo', start, end)
    tweets_list += tweets

df1 = pd.DataFrame(tweets_list, columns=['Text'])
df1['clean_text']= df1.apply(lambda row: clean_tweet(row['Text']), axis=1)
df1['sentiment_score']= df1.apply(lambda row: sentiment(row['clean_text']), axis=1)
df1['senti_label']= df1.apply(lambda row: sentiment_label(row['sentiment_score']), axis=1)

df1.shape

(1060624, 4)


In [10]:
df1.sentiment_score.mean()

0.1348961834119247


In [11]:
result = df1.groupby('senti_label').count()['Text']
result = result.reset_index()
total = df1.shape[0]

print('There are',total, ' tweets overall during the span about Cristiano Ronaldo.')

for i in range(5):
    value = result['Text'][i]
    label = result['senti_label'][i]
    rate = round(100*value/total,3)
    print(value, 'tweets (',rate, '% ) are labeled as',label)

There are 1060624  tweets overall during the span about Cristiano Ronaldo.
6901 tweets ( 0.651 % ) are labeled as NEGATIVE
73710 tweets ( 6.95 % ) are labeled as POSITIVE
57310 tweets ( 5.403 % ) are labeled as negative
669959 tweets ( 63.166 % ) are labeled as neutral
252744 tweets ( 23.83 % ) are labeled as positive
